# Fine-tuning GPT-2 for text classification
Stefan/Yuzhao Heng
Since Wed. Feb. 9th, 2022


Reproduce the results in paper [Zero-shot Text Classification With Generative Language Models](https://arxiv.org/abs/1912.10165),
since the authors didn't release the code.

Serve as infrastructure and baseline for project on efficient and accurate encoder for text classification with many labels.


## Setup



In [2]:
%load_ext autoreload
%autoreload 2


import torch
import transformers
from transformers import AutoTokenizer, AutoModel, AutoConfig, GPT2Tokenizer
from datasets import load_dataset
from icecream import ic


from util import *


transformers.set_seed(config('random-seed'))



## Prep Model & Dataset


In [5]:
dnm = 'ag_news'
dset = load_dataset(dnm)
# ic(dset)


# Models used in the paper, parameter sizes: 124M, 355M; Discrepancy with Open AI reported numbers?
model_nms = dict(small='gpt2', large='gpt2-medium')
model_nm = model_nms['small']
model = AutoModel.from_pretrained(model_nm)
tokenizer = AutoTokenizer.from_pretrained(model_nm)
# ic(tokenizer)

# def model_param_size(m: torch.nn.Module) -> str:
#    return fmt_num(sum(p.numel() for p in m.parameters()))
#
# ic(model_param_size(model))
# ic(model_param_size(AutoModel.from_pretrained('gpt2-medium')))


# ic(AutoConfig.from_pretrained(model_nm))



# s = "Which of these choices best describes the following document? : “ A pool For All Bodies ” , “ Lawmakers say they’d take pay cut, but they can’t ” , “ Raiders’ Gareon Conley faces civil suit ” , “ Proliﬁc cybercriminal suspected of spreading ransomware arrested by Polish Police [Europol] ”"
# # ic(tokenizer(s))
# ic(tokenizer.tokenize(s))





Using custom data configuration default
Reusing dataset ag_news (/Users/stefanh/.cache/huggingface/datasets/ag_news/default/0.0.0/bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548)


  0%|          | 0/2 [00:00<?, ?it/s]

ic| tokenizer: PreTrainedTokenizerFast(name_or_path='gpt2', vocab_size=50257, model_max_len=1024, is_fast=True, padding_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>'})
ic| AutoConfig.from_pretrained(model_nm): GPT2Config {
                                            "activation_function": "gelu_new",
                                            "architectures": [
                                              "GPT2LMHeadModel"
                                            ],
                                            "attn_pdrop": 0.1,
                                            "bos_token_id": 50256,
                                            "embd_pdrop": 0.1,
                                            "eos_token_id": 50256,
                                            "initializer_range": 0.02,
                                            "layer_norm_epsilon": 1e-05,
                                            "mo

['Which',
 'Ġof',
 'Ġthese',
 'Ġchoices',
 'Ġbest',
 'Ġdescribes',
 'Ġthe',
 'Ġfollowing',
 'Ġdocument',
 '?',
 'Ġ:',
 'ĠâĢ',
 'ľ',
 'ĠA',
 'Ġpool',
 'ĠFor',
 'ĠAll',
 'ĠB',
 'odies',
 'ĠâĢ',
 'Ŀ',
 'Ġ,',
 'ĠâĢ',
 'ľ',
 'ĠLaw',
 'makers',
 'Ġsay',
 'Ġthey',
 'âĢ',
 'Ļ',
 'd',
 'Ġtake',
 'Ġpay',
 'Ġcut',
 ',',
 'Ġbut',
 'Ġthey',
 'Ġcan',
 'âĢ',
 'Ļ',
 't',
 'ĠâĢ',
 'Ŀ',
 'Ġ,',
 'ĠâĢ',
 'ľ',
 'ĠRaiders',
 'âĢ',
 'Ļ',
 'ĠG',
 'are',
 'on',
 'ĠCon',
 'ley',
 'Ġfaces',
 'Ġcivil',
 'Ġsuit',
 'ĠâĢ',
 'Ŀ',
 'Ġ,',
 'ĠâĢ',
 'ľ',
 'ĠPro',
 'li',
 'ï',
 '¬',
 'ģ',
 'c',
 'Ġcyber',
 'criminal',
 'Ġsuspected',
 'Ġof',
 'Ġspreading',
 'Ġransomware',
 'Ġarrested',
 'Ġby',
 'ĠPolish',
 'ĠPolice',
 'Ġ[',
 'Euro',
 'pol',
 ']',
 'ĠâĢ',
 'Ŀ']